In [45]:
from string import ascii_lowercase
from string import ascii_uppercase
from more_itertools import locate
from collections import defaultdict
import re
import time
import json

In [46]:
def get_variants(s):
    
    vars = []
    
    if 'variants:' in s.lower():
        
        _ = s[s.lower().find('variants:') + len('variants:'):].strip()
        first_letter = _[0]

        if first_letter in ascii_uppercase:
            wrong_capitals = set(ascii_uppercase) - {first_letter}
            for ch in _:
                if (ch.isalpha() or (ch in {',',' '})) and (ch not in wrong_capitals):
                    vars.append(ch)
                else:
                    break
    if vars:
        return [v.lower().strip() for v in ''.join(vars).split(',') if v.strip()]
    else:
        return None
    
def get_origins(s):
    
    p1 = re.compile(r'\d{1}\:*\s*[a-zA-Z,\s\(\)]+:')
    p2 = re.compile(r'[\n|^]+[A-Z]+[ ]*\(*\w*\)*(?:,\s+[A-Z]+[\w\s]+)*:')
    
    origs = set()
    
    search_res = re.findall(p1, s)
    
    if search_res:
        for r in search_res:
            origs.update({orig.lower().strip() for orig in ''.join([c for c in r.split('(')[0].strip() if c.isalpha() or c in [',',' ']]).split(',') if orig.strip()})
    else:
        search_res = re.findall(p2, s)

        if search_res:
            for r in search_res:
                origs.update({o.lower() for o in {orig.split('(')[0].strip() for orig in r.replace(':','').split(',') if orig.istitle()} if o != 'References'})
    
    return list(origs) if origs else None

In [47]:
if __name__ == '__main__':
    
    t0 = time.time()
    
    with open('oxford_pages.txt','r', encoding='latin-1') as f:
        t = [line.strip() for line in f.readlines() if line.strip()]
    
    t = ['###' if ''.join(['.']*20) in s else s for s in t]
    
    brks = list(locate(t, lambda x: x == '###'))  
    
    names = []
    
    for i in range(len(brks)-1):
        
        name_info = defaultdict()
        
        name_info['name'] = t[brks[i]-1].lower().strip()
        
        name_desc = '\n'.join(t[brks[i]+1:brks[i+1]-1])
        
        name_info['variants'] = get_variants(name_desc)
        name_info['origins'] = get_origins(name_desc)
        
        names.append(name_info)
    
    print('done. {} names. elapsed time: {:.0f} min {:.0f} sec'.format(len(names), *divmod(time.time() - t0, 60)))
    json.dump(names, open('oxford_names.json','w'))  

done. 42540 names. elapsed time: 0 min 2 sec
